In [19]:
# Cell 1 – imports and ticker

import yfinance as yf
import pandas as pd
import numpy as np
import xlsxwriter

# Choose your ticker
ticker = "NVDA"   # change this to "AAPL", "TSLA", "TCS.NS", "BHP.AX", etc.

# yfinance Ticker object (used everywhere else)
tkr = yf.Ticker(ticker)


In [20]:
# Cell 2 – download financial statements (IN FULL DOLLARS)

# Annual income statement, balance sheet, and cash flow
fin = tkr.financials.T.copy()
bs  = tkr.balance_sheet.T.copy()
cf  = tkr.cashflow.T.copy()

# Sort rows oldest → newest
fin = fin.sort_index()
bs  = bs.sort_index()
cf  = cf.sort_index()




In [21]:
# Cell 3 – build historical FCF table (full dollars)

df = pd.DataFrame(index=fin.index)

# Revenue and EBIT
df["Revenue"] = fin.get("Total Revenue")
df["EBIT"]    = fin.get("Operating Income")

# Depreciation & Amortization
dep_cols = [c for c in cf.columns if "Depreciation" in c]
if dep_cols:
    df["D&A"] = cf[dep_cols[0]]
else:
    df["D&A"] = 0.0

# CapEx – Yahoo gives it as NEGATIVE cash flow; we want POSITIVE outflow
capex_cols = [c for c in cf.columns if "Capital Expenditure" in c]
if capex_cols:
    df["CapEx"] = -cf[capex_cols[0]]   # invert sign
else:
    df["CapEx"] = 0.0

# Net Working Capital = Total Current Assets - Total Current Liabilities
ca_candidates = [c for c in bs.columns if "Total Current Assets" in c]
cl_candidates = [c for c in bs.columns if "Total Current Liabilities" in c]

if ca_candidates and cl_candidates:
    ca_col = ca_candidates[0]
    cl_col = cl_candidates[0]
    df["NWC"] = (bs[ca_col] - bs[cl_col]).fillna(0)
else:
    df["NWC"] = 0.0

# ΔNWC
df["Delta_NWC"] = df["NWC"].diff().fillna(0)

# Tax rate (you can tune this)
tax_rate = 0.25

# NOPAT = EBIT * (1 - tax_rate)
df["NOPAT"] = df["EBIT"] * (1 - tax_rate)

# Free Cash Flow = NOPAT + D&A - CapEx - ΔNWC
df["FCF"] = df["NOPAT"] + df["D&A"] - df["CapEx"] - df["Delta_NWC"]



In [22]:
# Cell 4 – compute revenue CAGR and growth path

forecast_years = 5

hist_rev = df["Revenue"].dropna()

if len(hist_rev) >= 2:
    cagr = (hist_rev.iloc[-1] / hist_rev.iloc[0]) ** (1 / (len(hist_rev) - 1)) - 1
else:
    cagr = 0.10  # fallback

print("Historical revenue CAGR: ", round(cagr * 100, 2), "%")

# Let growth decline from CAGR to 2% over the forecast horizon
growth_rates = np.linspace(cagr, 0.02, forecast_years)
growth_rates


Historical revenue CAGR:  69.25 %


array([0.69254709, 0.52441032, 0.35627354, 0.18813677, 0.02      ])

In [23]:
# Cell 5 – forecast revenues and FCF (full dollars)

# Forecast revenue path
rev_forecast = []
current_rev = hist_rev.iloc[-1]

for g in growth_rates:
    current_rev *= (1 + g)
    rev_forecast.append(current_rev)

rev_forecast = np.array(rev_forecast)

# Build forecast DataFrame
forecast = pd.DataFrame(index=[f"Year {i+1}" for i in range(forecast_years)])
forecast["Revenue"] = rev_forecast

# Historical ratios
ebit_margin = (df["EBIT"] / df["Revenue"]).iloc[-1]
da_ratio    = (df["D&A"] / df["Revenue"]).mean()
capex_ratio = (df["CapEx"] / df["Revenue"]).mean()
nwc_ratio   = (df["Delta_NWC"] / df["Revenue"]).mean()

# Apply ratios
forecast["EBIT"]       = forecast["Revenue"] * ebit_margin
forecast["D&A"]        = forecast["Revenue"] * da_ratio
forecast["CapEx"]      = forecast["Revenue"] * capex_ratio
forecast["Delta_NWC"]  = forecast["Revenue"] * nwc_ratio

forecast["NOPAT"] = forecast["EBIT"] * (1 - tax_rate)
forecast["FCF"]   = forecast["NOPAT"] + forecast["D&A"] - forecast["CapEx"] - forecast["Delta_NWC"]

forecast


,Revenue,EBIT,D&A,CapEx,Delta_NWC,NOPAT,FCF
Year 1,2.208723e+11,1.378630e+11,7.722826e+09,8.092900e+09,0.0,1.033973e+11,1.030272e+11
Year 2,3.367000e+11,2.101598e+11,1.177276e+10,1.233690e+10,0.0,1.576199e+11,1.570557e+11
Year 3,4.566574e+11,2.850342e+11,1.596708e+10,1.673221e+10,0.0,2.137757e+11,2.130105e+11
Year 4,5.425714e+11,3.386596e+11,1.897107e+10,1.988016e+10,0.0,2.539947e+11,2.530857e+11
Year 5,5.534228e+11,3.454328e+11,1.935049e+10,2.027776e+10,0.0,2.590746e+11,2.581474e+11


In [24]:
# Cell 6 – WACC and terminal assumptions

info = tkr.info

debt = info.get("totalDebt", 0) or 0
cash = info.get("totalCash", 0) or 0
net_debt = debt - cash

# Assumptions (you can tune these)
risk_free = 0.04            # 4%
equity_risk_premium = 0.05  # 5%
beta = info.get("beta", 2.27)

cost_of_equity = risk_free + beta * equity_risk_premium  # CAPM

cost_of_debt_pre_tax = 0.03
tax_rate = 0.15
cost_of_debt_after_tax = cost_of_debt_pre_tax * (1 - tax_rate)

market_cap = info.get("marketCap", 1)
E = market_cap
D = max(net_debt, 0)
V = E + D

w_e = 0.95
w_d = 0.05

wacc = w_e * cost_of_equity + w_d * cost_of_debt_after_tax

terminal_growth = 0.03  # 2% long-term growth
# Exit multiple – from agent Assumptions
exit_multiple = 35.0          # 35x EBITDA

cost_of_equity
wacc, terminal_growth, net_debt


(0.147765, 0.03, -49786000384)

In [25]:
# Cell 7 – DCF valuation (Gordon + Exit multiple)

fcf_vals = forecast["FCF"].values
years = np.arange(1, forecast_years + 1)

# Discount factors
disc_factors = 1 / ((1 + wacc) ** years)

# PV of forecast FCFs
pv_fcfs = (fcf_vals * disc_factors).sum()

# --- Gordon Growth Terminal Value ---
fcf_last = fcf_vals[-1]
terminal_value_gordon = fcf_last * (1 + terminal_growth) / (wacc - terminal_growth)
pv_terminal_gordon = terminal_value_gordon / ((1 + wacc) ** forecast_years)

enterprise_value_gordon = pv_fcfs + pv_terminal_gordon

# --- Exit Multiple Terminal Value ---
forecast["EBITDA"] = forecast["EBIT"] + forecast["D&A"]

exit_multiple = 35.0  # you can change this

terminal_ebitda = forecast["EBITDA"].iloc[-1]
terminal_value_exit = terminal_ebitda * exit_multiple
pv_terminal_exit = terminal_value_exit / ((1 + wacc) ** forecast_years)

enterprise_value_exit = pv_fcfs + pv_terminal_exit

enterprise_value_gordon, enterprise_value_exit


(np.float64(1758798852974.0413), np.float64(7034999458569.277))

In [26]:
# Cell 8 – Equity value and per-share valuation

shares = info.get("sharesOutstanding", 1)

equity_value_gordon = enterprise_value_gordon - net_debt
equity_value_exit   = enterprise_value_exit   - net_debt

value_per_share_gordon = equity_value_gordon / shares
value_per_share_exit   = equity_value_exit   / shares

print("=== Gordon Growth Method ===")
print("Enterprise value (Gordon): {:,.0f}".format(enterprise_value_gordon))
print("Equity value (Gordon):     {:,.0f}".format(equity_value_gordon))
print("Value per share (Gordon):  ${:,.2f}".format(value_per_share_gordon))

print("\n=== Exit Multiple Method ===")
print("Enterprise value (Exit):   {:,.0f}".format(enterprise_value_exit))
print("Equity value (Exit):       {:,.0f}".format(equity_value_exit))
print("Value per share (Exit):    ${:,.2f}".format(value_per_share_exit))


=== Gordon Growth Method ===
Enterprise value (Gordon): 1,758,798,852,974
Equity value (Gordon):     1,808,584,853,358
Value per share (Gordon):  $74.41

=== Exit Multiple Method ===
Enterprise value (Exit):   7,034,999,458,569
Equity value (Exit):       7,084,785,458,953
Value per share (Exit):    $291.49
